In [2]:
import socket
import requests
import json
import random

# Metodo para obtener las preguntas de la API
def get_questions():
    response = requests.get("https://opentdb.com/api.php?amount=10")
    get_json = json.loads(response.text)
    results = get_json["results"]
   
    for i in range(len(results)):
        results[i]["question"] = results[i]["question"].replace("&quot;", "\"")
        results[i]["question"] = results[i]["question"].replace("&#039;", "'")
        results[i]["question"] = results[i]["question"].replace("&amp;", "&")
        results[i]["question"] = results[i]["question"].replace("&eacute;", "é")
        results[i]["question"] = results[i]["question"].replace("&aacute;", "á")
        results[i]["question"] = results[i]["question"].replace("&ntilde;", "ñ")
        results[i]["question"] = results[i]["question"].replace("&ouml;", "ö")
        results[i]["question"] = results[i]["question"].replace("&uuml;", "ü")
        results[i]["question"] = results[i]["question"].replace("&auml;", "ä")

    return get_json["results"]

# Metodo para enviar las preguntas al cliente
def send_question(socket_server, client_address, question):
    message = f"{question}"
    socket_server.sendto(str.encode(message), client_address)
    
# Metodo para comprobar que la respuesta del cliente es la correcta
def handle_client_response(client_response, correct_answer):
    return client_response.lower() == correct_answer.lower()

def main():

    # Configurar el socket del servidor
    server_address = ("127.0.0.1", 6789)
    socket_server = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
    socket_server.bind(server_address)
    print("UDP server up and listening...\n")

    # Ahora llamamos al metodo get_questions() para obtener las preguntas de la API
    questions = get_questions()

    # Variable para llevar la cuenta de aciertos
    success = 0

    while True:

        bytes_rx = socket_server.recvfrom(1024)
        mensaje = bytes_rx[0]
        client_address = bytes_rx[1]
        print("Cliente conectado")

        for i in range(3):
            # Enviar pregunta a los clientes           
            pregunta =  "Pregunta " + str(i+1) +":\n" + questions[i]["question"]

            respuestas = questions[i]["incorrect_answers"] 
            respuestas.append(questions[i]["correct_answer"])
            random.shuffle(respuestas)
           
            respuestas_bonitas =  ""
            for j in range(len(respuestas)):
                respuestas_bonitas += str(j+1) + ". " + respuestas[j] + "\n"

            pregunta += "\n" + "Opciones: \n" + respuestas_bonitas + "\n" + "Escribe la respuesta correcta: " 

            send_question(socket_server,client_address,pregunta)
            print("\nPregunta " + str(i+1) +":")
            print(questions[i]["question"])
            print("Respuesta correcta:")
            print(questions[i]["correct_answer"])


            # Aqui compruebo que recibo la respuesta
            bytes_rx = socket_server.recvfrom(1024)
            mensaje = bytes_rx[0] #guardo la respuesta en la variable mensaje

            # Ahora compruebo que la respuesta es la correcta
            respuesta_correcta = questions[i]["correct_answer"]

            if handle_client_response(mensaje.decode(), respuesta_correcta):
                msg = "Respuesta correcta" 
                success += 1
            else:
                msg = "Respuesta incorrecta" 
            
            socket_server.sendto(str.encode(msg), client_address)
            is_ok = socket_server.recvfrom(1024)
            print("Respuesta del cliente:\n" + mensaje.decode())
            if (is_ok[0].decode() != "OK"):
                print("algo ha ido mal")
                break
        
        break
    if success == 1 : 
        finalizado = "\n¡FIN DEL JUEGO!\n" + "Has respondido " + str(success) + " pregunta correctamente."
    else:
        finalizado = "\n¡FIN DEL JUEGO!\n" + "Has respondido " + str(success) + " preguntas correctamente."

    socket_server.sendto(str.encode(finalizado), client_address)
    socket_server.close()

if __name__ == "__main__":
    main()

UDP server up and listening...

Cliente conectado

Pregunta 1:
Which American civilization is the source of the belief that the world would end or drastically change on December 21st, 2012?
Respuesta correcta:
The Mayans
Respuesta del cliente:
The Mayans

Pregunta 2:
What Nickelodeon game show featured a house on the set that the contestants would ransack to find hidden objects?
Respuesta correcta:
Finders Keepers
Respuesta del cliente:
Double Dare
